# Dataset Formation (For testing purposes only using android dataset and using only 5000 postive pairs and random 5000 negative pairs)

## Imports

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 1.5MB 23.0MB/s 
     |████████████████████████████████| 1.2MB 56.8MB/s 
     |████████████████████████████████| 2.9MB 61.1MB/s 
     |████████████████████████████████| 890kB 30.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=483c96a567ec7424968a3e16644e688aa2835b1f364580bb1c8b0d8ab0d35a91
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e941b9e548e87f3b5190fc4398c7742a5d4692a5ae538f0e54b8eddb4a0a5f47
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
!pip install unidecode
!pip install word2number
!pip install gensim

     |████████████████████████████████| 245kB 18.0MB/s 
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5588 sha256=d295c197c9d7dea8c7cb6da3a7772e37da876f7ae3368cc918ba43e4427ff68e
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
Successfully built word2number


In [ ]:
! sudo apt install openjdk-8-jdk
! sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
! pip install language-check
! pip install pycontractions

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded, 15 newly installed, 0 to remove and 16 not upgraded.
Need to get 43.4 MB of archives.
After this 

In [ ]:
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
from pycontractions import Contractions
import gensim.downloader as api
import en_core_web_sm
import re
import json
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import random
import math

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sentence_transformers import models, losses, util, SentenceTransformer, SentencesDataset, InputExample, evaluation
from sentence_transformers.cross_encoder import CrossEncoder
from torch import nn
from torch.utils.data import DataLoader
import torch

%matplotlib notebook

# No need to run, if a script has run once

##Reading Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Data/android/android_linked.json','rb') as f:
    qr = pd.read_json(f,orient='table')

In [ ]:
qr.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 8797 entries, 31.0 to 218179.0
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AcceptedAnswerId  3528 non-null   float64
 1   AnswerCount       8797 non-null   float64
 2   AnswerIds         3053 non-null   object 
 3   Attachments       897 non-null    object 
 4   OS                6642 non-null   object 
 5   PostTypeId        8797 non-null   float64
 6   Related           3053 non-null   object 
 7   Score             8797 non-null   float64
 8   Tags              8797 non-null   object 
 9   Text              8797 non-null   object 
 10  Title             8797 non-null   object 
 11  Linked            8797 non-null   object 
dtypes: float64(4), object(8)
memory usage: 893.4+ KB


##Text Preprocessing

###Funcitonal Block

In [ ]:
nlp = en_core_web_sm.load()

# Choose model accordingly for contractions function
model = api.load("glove-twitter-25")
# model = api.load("glove-twitter-100")
# model = api.load("word2vec-google-news-300")

cont = Contractions(kv_model=model)
cont.load_models()

# exclude words from spacy stopwords list
deselect_stop_words = ['no', 'not']
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False


def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())


def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text


def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = list(cont.expand_texts([text], precise=True))[0]
    return text


def text_preprocessing(text, accented_chars=True, contractions=False, 
                       convert_num=True, extra_whitespace=True, 
                       lemmatization=True, lowercase=True, punctuations=False,
                       remove_html=True, remove_num=False, special_chars=False, 
                       stop_words=True):
    """preprocess text with default option set to true for all steps"""
    if remove_html == True: #remove html tags
        text = strip_html_tags(text)
    if extra_whitespace == True: #remove extra whitespaces
        text = remove_whitespace(text)
    if accented_chars == True: #remove accented characters
        text = remove_accented_chars(text)
    if contractions == True: #expand contractions
        text = expand_contractions(text)
    if lowercase == True: #convert all characters to lowercase
        text = text.lower()
    
    # return text

    doc = nlp(text) #tokenise text

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if stop_words == True and token.is_stop and token.pos_ != 'NUM': 
            flag = False
        # remove punctuations
        if punctuations == True and token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        # remove special characters
        if special_chars == True and token.pos_ == 'SYM' and flag == True: 
            flag = False
        # remove numbers
        if remove_num == True and (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        # convert number words to numeric numbers
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            try:
                edit = w2n.word_to_num(token.text)
            except:
                a=1
                # print('error in converting number words to numeric numbers',token.text)
        # convert tokens to base form
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list 
        if edit != "" and flag == True:
            clean_text.append(edit)      
    return clean_text



[========================================----------] 80.8% 84.7/104.8MB downloaded

In [ ]:
def lts(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += str(ele) + ' '  
    
    # return string   
    return str1  

###Iterating over all exmaples

In [ ]:
cnt=0
for i in qr.index.values:
    qr.loc[i,'Title'] = lts(text_preprocessing(qr.loc[i,'Title']))
    qr.loc[i,'Text'] = lts(text_preprocessing(qr.loc[i,'Text']))
    qr.loc[i,'Tags'] = lts(qr.loc[i,'Tags'])
    cnt+=1
    if(cnt%100==0):
        print(cnt)

5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700


In [ ]:
qr.to_json('/content/drive/MyDrive/sbert_supervised/android_l_nosw.json',orient='table')

##Positive Pair Generation 

In [ ]:
with open('/content/drive/MyDrive/sbert_supervised/android_nosw.json','r') as f:
    qr = pd.read_json(f,orient='table')

In [ ]:
def lts(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += str(ele) + ' '  
    
    # return string   
    return str1  

In [ ]:
qr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17132 entries, 31 to 219026
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AcceptedAnswerId  6631 non-null   float64
 1   AnswerCount       17132 non-null  float64
 2   AnswerIds         17132 non-null  object 
 3   Attachments       4246 non-null   object 
 4   OS                4246 non-null   object 
 5   PostTypeId        17132 non-null  int64  
 6   Related           17132 non-null  object 
 7   Score             17132 non-null  int64  
 8   Tags              17132 non-null  object 
 9   Text              17132 non-null  object 
 10  Title             17132 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.6+ MB


In [ ]:
data_pos = list()
cnt = 0
related_not_found = 0
al_pos = list()
for i in qr.index.values:
    # if(cnt==num_pairs):
    #     break
    if(len(qr.loc[i,'Related'])!=0):
        for j in qr.loc[i,'Related']:
            if( ( [j,i] in al_pos or [i,j] in al_pos) != 1 ):
                try:
                    entry=dict()
                    entry['id1'] = i
                    entry['id2'] = j
                    entry['title1'] = qr.loc[i,'Title']
                    entry['title2'] = qr.loc[j,'Title']
                    entry['tag1'] = lts(qr.loc[i,'Tags'])
                    entry['tag2'] = lts(qr.loc[j,'Tags'])
                    entry['text1'] = qr.loc[i,'Text']
                    entry['text2'] = qr.loc[j,'Text']
                    entry['score'] = float(1.0)
                    cnt+=1
                    data_pos.append(entry)
                    al_pos.append([ i , j ])
                    # break
                except:
                    related_not_found+=1
                    # print('Related not found,parent id: ',i,' id:',j,'overall count: ',related_not_found)
print(related_not_found)
print(len(data_pos))

26966
78702


##Negative pair Generation ( 50:50 dataset ratio)

In [ ]:
num_pairs = len(data_pos)
data_neg = list()
al_acc = list()
for i in range(0,num_pairs):
    id1 = random.choice(qr.index.values)
    id2 = random.choice(qr.index.values)
    while(id2 in qr.loc[id1,'Related'] or id1 in qr.loc[id2,'Related'] or [id1,id2] in al_acc or [id2,id1] in al_acc):
         id1 = random.choice(qr.index.values)
         id2 = random.choice(qr.index.values)
    entry=dict()
    entry['id1'] = id1
    entry['id2'] = id2
    entry['title1'] = qr.loc[id1,'Title']
    entry['title2'] = qr.loc[id2,'Title']
    entry['tag1'] = lts(qr.loc[id1,'Tags'])
    entry['tag2'] = lts(qr.loc[id2,'Tags'])
    entry['text1'] = qr.loc[id1,'Text']
    entry['text2'] = qr.loc[id2,'Text']
    entry['score'] = float(0.0)
    data_neg.append(entry)
    al_acc.append([id1,id2])
print(len(data_neg))

78702


##Modelling

###Creating Train and Dev data

####Preprocessing all the title, tag, text and combining it in the order -> title + tag + text (Keep in mind, that bert input vector length is 512) and the spliting the overall dataset into train and dev

In [ ]:
data = list()
score = list()
for i in range(0,num_pairs):
    s1_pos = data_pos[i]['title1']+data_pos[i]['text1']
    s2_pos = data_pos[i]['title2']+data_pos[i]['text2']
    data.append([s1_pos,s2_pos])
    score.append(float(1))
    s1_pos = data_neg[i]['title1']+data_neg[i]['text1']
    s2_pos = data_neg[i]['title2']+data_neg[i]['text2']
    data.append([s1_pos,s2_pos])
    score.append(float(0))

###Saving the list

In [ ]:
import pickle
with open('/content/drive/MyDrive/sbert_supervised/data_android_wt.txt','wb') as d:
    pickle.dump(data,d)
with open('/content/drive/MyDrive/sbert_supervised/score_android_wt.txt','wb') as s:
    pickle.dump(score,s)

#For Rerun, run from here

###Loading the list

In [ ]:
from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
with open('/content/drive/MyDrive/sbert_supervised/data_android.txt','rb') as d:
    data = pickle.load(d)
with open('/content/drive/MyDrive/sbert_supervised/score_android.txt','rb') as s:
    score = pickle.load(s)
print(len(data),len(score))

157404 157404


In [ ]:
# for i in range(0,len(score)):
#     if(score[i]==0.0):
#         score[i]=float(np.random.uniform(low=-1.0, high=-0.9))
#     if(score[i]==1.0):
#         score[i] = float(np.random.uniform(low=0.9,high=1.0))

for i in range(0,len(score)):
    score[i] = int(score[i])
print(type(score[0]))

<class 'int'>


In [ ]:
data,score = shuffle(data,score)
train_data,dev_data,train_score,dev_score = train_test_split(data,score,test_size=0.1,random_state=13)
train_data,test_data,train_score,test_score = train_test_split(train_data,train_score,test_size=0.2,random_state=7)

In [ ]:
print(torch.cuda.is_available())

True


###Creating Model

In [ ]:
word_embedding_model = models.Transformer('bert-base-uncased',max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
train_examples = list()
for i in range(0,len(train_data)):
    train_examples.append(InputExample(texts=[train_data[i][0],train_data[i][1]], label=train_score[i]))

dev_s1=list()
dev_s2 = list()
for i in range(0,len(dev_data)):
    dev_s1.append(dev_data[i][0])
    dev_s2.append(dev_data[i][1])
test_s1=list()
test_s2 = list()
for i in range(0,len(test_data)):
    test_s1.append(test_data[i][0])
    test_s2.append(test_data[i][1])

In [ ]:
batch_size=8
num_epochs = 5
model_path = '/content/drive/MyDrive/hello'

###Training Model

In [ ]:
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
train_loss =  losses.SoftmaxLoss(model=model, sentence_embedding_dimension = model.get_sentence_embedding_dimension(),num_labels = 2 )

In [ ]:
evaluator_dev = evaluation.BinaryClassificationEvaluator(dev_s1, dev_s2, dev_score)
evaluator_test = evaluation.BinaryClassificationEvaluator(test_s1, test_s2, test_score)

In [ ]:
warmup_steps = math.ceil(len(train_dataset) * num_epochs / batch_size * 0.1) #10% of train data for warm-up

###Fitting Model

In [ ]:
x = list()
y = list()
def callback_model(score,epoch,steps):
    if(steps==-1):
        steps=0
        epoch+=1
    x.append(float(epoch + steps/89590))
    y.append(float(score))
    print('Score after epoch ', str(epoch),' and steps ', str(steps),'is ',str(score))

In [ ]:
for i in enumerate (train_dataloader):
    print('Hello')

TypeError: ignored

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator_dev,
          epochs=num_epochs,
          evaluation_steps=4000,
          warmup_steps = warmup_steps,
          output_path=model_path,
          output_path_ignore_not_empty=True,
          use_amp = True,
          callback = callback_model
          )

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(evaluator_test)

In [ ]:
plt.plot(x,y)
plt.show()

In [ ]:
with open('./sbert_softmax/Accuracy_score.txt','wb') as f:
    pickle.dump(x,f)
    pickle.dump(y,f)

In [ ]:
plt.savefig('/content/drive/MyDrive/sbert_supervised/final_accuarcy.png')